# Работа со строковыми значениями

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for k, v in obj.items():
    print(f"{k:<15} = {v}")


home_page       = https://github.com/pypa/sampleproject
keywords        = sample setuptools development
license         = MIT


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
import re
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
for s in obj:
    match = re.search(r'\d{1,2}-\d{1,2}', s)
    if match:
        print(match.group())

19-1
20-4
20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [5]:
text = "Написать регулярное выражение,которое позволит найти номера групп студентов."
words = re.findall(r'\b\w+\b', text)
print(words)

['Написать', 'регулярное', 'выражение,которое', 'позволит', 'найти', 'номера', 'групп', 'студентов.']


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [9]:
recipes = pd.read_csv('recipes_sample.csv')

for _, row in recipes.sample(5).iterrows():
    print("id: {:<10} minutes: {:<10}".format(row['id'], row['minutes']))

id: 269680     minutes: 35        
id: 317716     minutes: 75        
id: 216167     minutes: 30        
id: 215254     minutes: 5         
id: 35509      minutes: 45        


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [8]:
import csv
import xml.etree.ElementTree as ET

def show_info(recipe_id):
    with open('recipes_sample.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        # Поиск рецепта по идентификатору
        for row in reader:
            if row['id'] == recipe_id:
                # Получение информации о рецепте
                recipe_name = row['name'].title()
                author_id = row['contributor_id']
                cook_time = int(row['minutes'])
                break

    with open('steps_sample.xml', encoding='utf-8') as xmlfile:
        root = ET.fromstring(xmlfile.read())
        steps = root.findall(".//recipe[@id='" + recipe_id + "']/recipetext")      # Поиск шагов рецепта по идентификатору

    # Формирование строки
    result = recipe_name + "\n\n"
    for i, step in enumerate(steps):
        result += f"{i+1}. {step.text}\n"
    result += "----------\n"
    result += f"Автор: {author_id}\n"
    result += f"Среднее время приготовления: {cook_time} минут\n"
    return result

print(show_info('170895'))

Leeks And Parsnips  Sauteed Or Creamed

----------
Автор: 8377
Среднее время приготовления: 27 минут



## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [31]:
import re
import xml.etree.ElementTree as ET

from bs4 import BeautifulSoup
with open('steps_sample.xml', mode = 'r') as file:
    step_xml = BeautifulSoup(file, 'xml')


pattern = re.compile(r'\d+\s(?:hour|hours|minute|minutes)')
for i in step_xml.find_all('recipe'):
    if int(i.find('id').get_text()) == 25082:
        for j in i.find_all('step'):
            res = pattern.findall(j.get_text())
            if res:
                print(res)

['20 minute']
['10 minute']
['2 hour']
['10 minute']
['20 minute', '30 minute']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [27]:
import pandas as pd
import re

recipes = pd.read_csv('recipes_sample.csv')

matching_recipes = recipes['description'].str.match('^this[\w\s]*\.{3}\s*,?\s*but')
num_matching_recipes = matching_recipes.sum()

print(f'Количество рецептов с подходящим описанием: {num_matching_recipes}\n')
print('Примеры описаний:')
recipes[matching_recipes == True]

Количество рецептов с подходящим описанием: 2

Примеры описаний:


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18466,my own best bbq ed baked beans,120245,80,113941,2005-05-01,NaN,"this makes a lot...but...""one for the potluck ...",NaN
21958,quick n easy eclair cake,43366,270,56447,2002-10-16,NaN,this dessert dish sounds a little strange...bu...,5.0


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [38]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')
recipes

steps = recipes[recipes['id'] == 72367]['description']
steps = steps.str.replace(r'\s*/\s*', '/') # Убираем пробелы до и после символа дроби
print(steps)

2459    such a beautiful tart - just like from a europ...
Name: description, dtype: object


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [41]:
import nltk

recipes = pd.read_csv('recipes_sample.csv')
# Объединение текстов шагов в одну строку
text = ' '.join(recipes[recipes['id'] == 72367]['description'].values)

tokens = nltk.word_tokenize(text)# Разбиение текста на слова
words = [word.lower() for word in tokens if word.isalpha()]# Отбор только слов (исключение знаков препинания и т.п.)

unique_words = set(words)# Подсчет количества уникальных слов
count = len(unique_words)
print(f'Количество уникальных слов: {count}')


Количество уникальных слов: 16


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [10]:
import nltk
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

recipes['sentences'] = recipes['description'].apply(lambda x: len(nltk.sent_tokenize(str(x))))

top5 = recipes.sort_values('sentences', ascending=False).head(5) #ascending = False - сортировка по убыванию
print(top5)

                                                    name      id  minutes  \
18408       my favorite buttercream icing for decorating  334113       30   
481    alligator claws  avocado fritters  with chipot...  287008       45   
22566                          rich barley mushroom soup  328708       60   
16296  little bunny foo foo cake  carrot cake  with c...  316000       68   
6779                                       chocolate tea  205348        6   

       contributor_id   submitted  n_steps  \
18408          681465  2008-10-30     12.0   
481            765354  2008-02-19      NaN   
22566          221776  2008-10-03      NaN   
16296          689540  2008-07-27     14.0   
6779           428824  2007-01-14      NaN   

                                             description  n_ingredients  \
18408  this wonderful icing is used for icing cakes a...            NaN   
481    a translucent golden-brown crust allows the gr...            9.0   
22566  this is one of the best soup

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [11]:
def pos_tags(sentence):
    tags = nltk.pos_tag(nltk.word_tokenize(sentence))
    output = ""
    for word, tag in tags:
        output += f"{tag: <{5}} "
    output += "\n"
    for word, tag in tags:
        output += f"{word: <{5}} "
    return output

recipe = recipes[recipes['id'] == 241106]
description = recipe.iloc[0]["description"]

print(pos_tags(description))

DT    VBP   DT    RB    JJ    NN    NN    TO    VB    IN    DT    NN    WRB   JJ    NNS   VBP   JJ    NN    CC    NNS   VBG   IN    .     DT    VBG   VBN   VBZ   DT    NN    ,     CC    EX    VBP   RB    JJ    JJR   NNS   TO    VB    DT    NN    .     VB    IN    IN    DT    JJ    NNS   CC    RB    VB    IN    WP    VBZ   JJ    TO    PRP   .     
these are   a     really good  quick meal  to    make  in    the   summertime when  local farms have  fresh eggplant and   tomatoes coming in    .     the   topping given is    a     suggestion ,     but   there are   certainly many  more  ways  to    enjoy this  dish  .     start out   with  the   eggplant steaks and   then  top   with  what  sounds good  to    you   .     
